## 埋込モデル

### 準備

必要なライブラリのインストール

In [ ]:
%pip install -r ../requirements.txt

### 環境変数の設定

In [2]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

### 埋め込みモデル


In [3]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-multilingual-v3.0"
)

/data/ghq/github.com/codex-odyssey/llm-observability/venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


### Document Loader

`../docs/*.txt` に格納されているテキストデータを読み込み、LangChain の Document へ変換します。

In [4]:
import glob
from langchain.document_loaders import TextLoader

files = glob.glob("../app/docs/*.txt")
documents = []

for file in files:
    loader = TextLoader(file_path=file)
    document = loader.load()
    documents.extend(document)
documents

[Document(metadata={'source': '../app/docs/karubikuppa.txt'}, page_content='BB流カルビクッパの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 牛肉: 50g\n- 大根: 半分\n- 人参: 1本\n- シイタケ: 2-3個\n- ニンニク: 半分\n- もやし: 1袋\n- ニラ: 半分\n- ぜんまい: 40g\n    - BB流ポイント 「入れなくても良いですが、入れると@<ttb>{めちゃ旨}です！」\n- つゆ (*)\n    - 水: 800cc\n    - コチュジャン: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - ウェイパー: 大さじ2\n    - ごま油: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - 酒: 大さじ2\n    - 醤油: 大さじ2\n    - 砂糖: 大さじ2\n- 塩コショウ:適量\n- 卵: 2個\n\nごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（*）をいれ、ニラともやし以外を入れてひと煮立ちさせます。\n最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたらBB流カルビクッパの完成です！\nなお、卵をふんわりさせるコツはつゆを限界まで沸騰させることです！\n'),
 Document(metadata={'source': '../app/docs/arani.txt'}, page_content='BB流！200円で作れる真鯛のアラの塩煮の作り方\n\n用意するもの\n\n- フライパン\n- アルミホイル\n- しょうが1欠片\n- 真鯛のアラ（血とかは洗っておく）\n- 塩\n- 水\n\n作り方は以下の通りです。\n\n1．フライパンに水（落とし蓋をするのでやや少なめ）を生姜のスライスと一緒に入れる\n2．沸騰したら塩を小さじ2（スープがしょっぱかったので少なくてもいいかも）を入れて溶かしてから真鯛のアラを入れる。沸騰してから入れると前処理無しで臭みなし\n3．アルミホイルで落とし蓋する\n4．火が通るまで弱火で適当に煮る\n5．(゜Д゜)ｳﾏｰ\n\n

### Text Splitters

In [6]:
from langchain_community.document_loaders import WebBaseLoader

langchain_doc = WebBaseLoader("https://python.langchain.com/docs/concepts/").load()

len(langchain_doc[0].page_content)


82912

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

langchain_docs = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(langchain_doc)
len(langchain_docs)

125

In [8]:
from langchain_community.vectorstores import FAISS

langchain_db = FAISS.from_documents(langchain_docs, CohereEmbeddings(model="embed-multilingual-v3.0"))
result = langchain_db.similarity_search("Please tell me how to set up the PR to separate it into minor and patch versions ?")
result

[Document(metadata={'source': 'https://python.langchain.com/docs/concepts/', 'title': 'Conceptual guide | 🦜️🔗 LangChain', 'description': 'This section contains introductions to key parts of LangChain.', 'language': 'en'}, page_content='middle" effectHow to split Markdown by HeadersHow to merge consecutive messages of the same typeHow to add message historyHow to migrate from legacy LangChain agents to LangGraphHow to retrieve using multiple vectors per documentHow to pass multimodal data directly to modelsHow to use multimodal promptsHow to create a custom Output ParserHow to use the output-fixing parserHow to parse JSON outputHow to retry when a parsing error occursHow to parse XML outputHow to parse YAML outputHow to use the Parent Document RetrieverHow to use LangChain with different Pydantic versionsHow to add chat historyHow to get a RAG application to add citationsHow to do per-user retrievalHow to get your RAG application to return sourcesHow to stream results from your RAG appl

### Vector Store

自然言語を用いた類似度検索を行います。

In [9]:
vector_store = FAISS.from_documents(documents, embeddings)
result = vector_store.similarity_search(query="カルビクッパ",k=1)
result

[Document(metadata={'source': '../app/docs/karubikuppa.txt'}, page_content='BB流カルビクッパの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 牛肉: 50g\n- 大根: 半分\n- 人参: 1本\n- シイタケ: 2-3個\n- ニンニク: 半分\n- もやし: 1袋\n- ニラ: 半分\n- ぜんまい: 40g\n    - BB流ポイント 「入れなくても良いですが、入れると@<ttb>{めちゃ旨}です！」\n- つゆ (*)\n    - 水: 800cc\n    - コチュジャン: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - ウェイパー: 大さじ2\n    - ごま油: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - 酒: 大さじ2\n    - 醤油: 大さじ2\n    - 砂糖: 大さじ2\n- 塩コショウ:適量\n- 卵: 2個\n\nごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（*）をいれ、ニラともやし以外を入れてひと煮立ちさせます。\n最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたらBB流カルビクッパの完成です！\nなお、卵をふんわりさせるコツはつゆを限界まで沸騰させることです！\n')]

### Retriever

Vector StoreからRetrieverの生成

In [10]:
retriever = vector_store.as_retriever()
retriever.invoke("カルビクッパ")

[Document(metadata={'source': '../app/docs/karubikuppa.txt'}, page_content='BB流カルビクッパの作り方です。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 牛肉: 50g\n- 大根: 半分\n- 人参: 1本\n- シイタケ: 2-3個\n- ニンニク: 半分\n- もやし: 1袋\n- ニラ: 半分\n- ぜんまい: 40g\n    - BB流ポイント 「入れなくても良いですが、入れると@<ttb>{めちゃ旨}です！」\n- つゆ (*)\n    - 水: 800cc\n    - コチュジャン: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - ウェイパー: 大さじ2\n    - ごま油: 大さじ2\n        - BB流ポイント「入れれば入れるだけ美味しい！」\n    - 酒: 大さじ2\n    - 醤油: 大さじ2\n    - 砂糖: 大さじ2\n- 塩コショウ:適量\n- 卵: 2個\n\nごま油でニンニクを炒め、香りが出たら牛肉を炒めます。つゆ（*）をいれ、ニラともやし以外を入れてひと煮立ちさせます。\n最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたらBB流カルビクッパの完成です！\nなお、卵をふんわりさせるコツはつゆを限界まで沸騰させることです！\n'),
 Document(metadata={'source': '../app/docs/spice-curry.txt'}, page_content='BB流のスパイスカレーのレシピです。\n\nまずは、以下の材料を用意します。なお、材料は2人分です。\n\n- 鶏モモ肉皮なし: 200g\n- たまねぎ1/2: 110g\n- にんにく: 4g（チューブ8㎝）\n- しょうが: 4g（チューブ8㎝）\n- 塩: 2g\n- オリーブ油: 8ｇ\n- トマト缶1/3: 60g\n- カレー粉: 5g\n- ガラムマサラ: 1g\n- ケチャップ: 18g\n- ウスターソース: 18g\n\n作り方は以下の通りです。\n\n1. 玉ねぎを粗みじんに切る\n2. 鶏モモは皮を外し一口大に切る\n3. 鍋に

WikipediaRetriever

In [11]:
from langchain_community.retrievers import WikipediaRetriever

retriever = WikipediaRetriever()
docs = retriever.invoke("Mobile Suit Gundam SEED Freedom")
docs[0].page_content[:50]

'Mobile Suit Gundam SEED Freedom (Japanese: 機動戦士ガンダ'

### BB流のレシピを教えてくれるアプリケーション

In [60]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(
    model="command-r-plus"
)
retriever = vector_store.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
あなたは有能なアシスタントです。
以下のコンテキストに基づいて質問に対する回答を作成してください。
分からない場合は分からないと回答してください。

## コンテキスト

{context}

## 質問

{question}
""")

chain = (
    {"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream("カルビクッパってどうやって作るのでしょうか？")

for chunk in response:
    print(chunk, end="")


BB流カルビクッパの作り方は、以下の通りです。

材料（2人分）
- 牛肉: 50g
- 大根: 半分
- 人参: 1本
- シイタケ: 2-3個
- ニンニク: 半分
- もやし: 1袋
- ニラ: 半分
- ぜんまい: 40g（オプション）
- つゆ:
  - 水: 800cc
  - コチュジャン: 大さじ2
  - ウェイパー: 大さじ2
  - ごま油: 大さじ2
  - 酒: 大さじ2
  - 醤油: 大さじ2
  - 砂糖: 大さじ2
- 塩コショウ: 適量
- 卵: 2個

作り方
1. ごま油でニンニクを炒め、香りが出たら牛肉を炒めます。
2. つゆをいれ、ニラともやし以外を入れてひと煮立ちさせます。
3. 最後にニラともやしを入れてある程度シャキシャキ感を残しつつ、卵をふんわりさせたら完成です。卵をふんわりさせるコツは、つゆを限界まで沸騰させることです。

ぜんまいは入れなくても良いですが、入れるとより一層美味しくなるそうです。また、ごま油とコチュジャンは「入れれば入れるだけ美味しい」とのことです。

以上です。お試しください！

### xQL完全ガイドについて教えてくれるアプリケーション

In [61]:
xql_doc = WebBaseLoader("https://techbookfest.org/product/vwEgK9fAmzRphNukv4E83P?productVariantID=b6iAh0AVyEs4hCUczPiy").load()
xql_db = FAISS.from_documents(xql_doc, CohereEmbeddings(model="embed-multilingual-v3.0"))
xql_retriever = xql_db.as_retriever()

In [36]:

chain = (
    {"question": RunnablePassthrough(), "context": xql_retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

response = chain.stream("xQL完全ガイドってなに？")

for chunk in response:
    print(chunk, end="")


xQL 完全ガイドは、Prometheus、Grafana Loki、Grafana Tempo などのモニタリング ツールで使用されるクエリ言語、PromQL、LogQL、TraceQL について解説したガイドブックです。これらのツールで計測したテレメトリー シグナルの中から、必要なメトリクス、ログ、トレースを探索する方法を学習できます。ガイドでは、各クエリ言語の解説に加え、参考問題やサンプル アプリ、Docker 環境が提供されており、読者が実際に手を動かしながら学ぶことができるようになっています。このガイドを通じて、システムのオブザーバビリティを高め、クラウドネイティブなアプリケーションの運用を効率化することが期待されます。